In [119]:
import math
import numpy as np
import yahoo_fin.stock_info as si
import requests
import pandas as pd
import datetime as dt
import os
import os.path
from dateutil.relativedelta import relativedelta
from py_vollib.black.greeks.analytical import delta
from Scripts.Utils import trading_utils
import math
from dateutil import parser
import yfinance as yf
from scipy import interpolate
from decimal import Decimal
from bisect import bisect

from ib_insync import *
util.startLoop()
from pathlib import Path
ib = IB()
ib.client.setConnectOptions('+PACEAPI')
ib.connect('127.0.0.1', 7497, clientId=20)

<IB connected to 127.0.0.1:7497 clientId=20>

In [123]:
def GetWeightsFromCsv(inputs_path):
    df = pd.read_csv(inputs_path)
    df = df.set_index(['symbols'])
    return df.T.to_dict()

def GetRelativeValueWeightsInputsPath():
    current_dir = Path(os.getcwd())
    project_dir = next(p for p in current_dir.parents if p.name == "Trading")
    relative_value_path = str(project_dir) + "\\Inputs\\relative_value_inputs_weights.csv"
    return relative_value_path

def GetRelativeValueInputsPath():
    current_dir = Path(os.getcwd())
    project_dir = next(p for p in current_dir.parents if p.name == "Trading")
    relative_value_path = str(project_dir) + "\\Inputs\\relative_value_inputs.csv"
    return relative_value_path

def GetImpliedVolsHistoricalPath():
    current_dir = Path(os.getcwd())
    project_dir = next(p for p in current_dir.parents if p.name == "Trading")
    implied_path = str(project_dir) + "\\HistoricalData\\stocks\\" + symbol + "\\implied.csv"
    return implied_path

def GetRealizedVolsHistoricalPath():
    current_dir = Path(os.getcwd())
    project_dir = next(p for p in current_dir.parents if p.name == "Trading")
    realized_path = str(project_dir) + "\\HistoricalData\\stocks\\" + symbol + "\\realized.csv"
    return realized_path

def GetInputs(inputs_path):
    df = pd.read_csv(inputs_path, header= None)
    df_dict = dict(zip(df[0], df[1]))
    print(df_dict)
    df_dict['time_lag_allowed'] = dt.timedelta(seconds = int(df_dict['time_lag_allowed_in_seconds']))
    return df_dict
def GetImpliedVolsDict(implied_vols_path, time_lag_allowed):
    df = pd.read_csv(implied_vols_path)
    df['data_collection_time'] = pd.to_datetime(df['data_collection_time'])
    last_time  = df.data_collection_time.max()
    current_time = dt.datetime.now()
    time_difference = current_time - last_time
    if time_difference > time_lag_allowed:
        raise ValueError("Last time stored " + str(last_time) + ". Time lag allowed : ", str(time_lag_allowed))
    df = df[df['data_collection_time'] == last_time]
    df = df.set_index(['expiry'])
    df = df[['0.85', '0.7', '0.5', '0.3', '0.15', 'tte']]
    implied_dict = df.T.to_dict()
    return implied_dict

def GetImpliedRatios(implied_ratios_path):
    df = pd.read_csv(implied_ratios_path)
    df = df.set_index('historical')
    return df.columns
def GetImpliedTheoPath():
    current_dir = Path(os.getcwd())
    project_dir = next(p for p in current_dir.parents if p.name == "Trading")
    implied_path = str(project_dir) + "\\Inputs\\implied_theo_ratios.csv"
    return implied_path

def GetTheoImpliedVolsDict(realized_vols_path, time_lag_allowed):
    df = pd.read_csv(realized_vols_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    last_time  = df.timestamp.max()
    current_time = dt.datetime.now()
    time_difference = current_time - last_time
    if time_difference > time_lag_allowed:
        raise ValueError("Last time stored " + str(last_time) + ". Time lag allowed : ", str(time_lag_allowed))
    df = df[df['timestamp'] == last_time]
    # print(df)
    implied_vols_theo_dict_raw = df.to_dict('records')[0]
    implied_theo_path = GetImpliedTheoPath()
    implied_theo_days_calculated = GetImpliedRatios(implied_theo_path)
    implied_vols_theo = {}
    # print(implied_vols_theo_dict_raw)
    for day in implied_theo_days_calculated:
        filter_string = day + "_day_implied"
        subset_dict =  {k:v for k,v in implied_vols_theo_dict_raw.items() if filter_string in k}
        implied_vols_theo[float(day)] = sum(subset_dict.values()) / len(subset_dict)
    return implied_vols_theo







In [124]:
inputs_path_weights = GetRelativeValueWeightsInputsPath()
weights_dict = GetWeightsFromCsv(inputs_path_weights)
inputs_path = GetRelativeValueInputsPath()
inputs_dict= GetInputs(inputs_path)
time_lag_allowed = inputs_dict['time_lag_allowed']

{'time_lag_allowed_in_seconds': 3000000}


In [125]:
for symbol in weights_dict:
    implied_vols_path = GetImpliedVolsHistoricalPath()
    implied_vols = GetImpliedVolsDict(implied_vols_path, time_lag_allowed)
    print(implied_vols)
    realized_vols_path = GetRealizedVolsHistoricalPath()
    theo_implied_vols = GetTheoImpliedVolsDict(realized_vols_path, time_lag_allowed)
    print(theo_implied_vols)
    for expiry in implied_vols:
        dte = implied_vols[expiry]['tte'] * 252
        theo_implied_vols_anchors = sorted(theo_implied_vols.keys())
        location = bisect(theo_implied_vols_anchors, dte)
        print(theo_implied_vols_anchors[location], dte)
        x1 = theo_implied_vols_anchors[location]
        y1 = theo_implied_vols[theo_implied_vols_anchors[location]]
        x2 = theo_implied_vols[location+1]
        y2 = theo_implied_vols[theo_implied_vols_anchors[location+1]]

        a,b = np.polyfit([x1,x2],[y1,y2])
        theo_at_dte = a * dte + b
        implied_vols[expiry]['ImpliedTheo'] = theo_at_dte
    return implied_vols


{'2022-09-30': {'0.85': 0.7626243694423094, '0.7': 0.7167426314276039, '0.5': 0.6799128023110422, '0.3': 0.6448345965468176, '0.15': 0.6227538584991178, 'tte': 0.0021825396825396}}
{1.0: 0.4042604276329835, 2.0: 0.3965751596355624, 4.0: 0.4042605364322307, 7.0: 0.3991369754092086, 11.0: 0.394013343861916, 16.0: 0.3888897828388938, 22.0: 0.38376636889009563, 29.0: 0.37864280786707355}
1.0 0.5499999999999793
{'2022-09-30': {'0.85': 0.6576749990277707, '0.7': 0.619674413418569, '0.5': 0.5854654576306698, '0.3': 0.5604661581841476, '0.15': 0.5462325623139002, 'tte': 0.0021825396825396}}
{1.0: 0.294014827368062, 2.0: 0.2955780737715727, 4.0: 0.294014823913524, 7.0: 0.29505697947844545, 11.0: 0.29609916806469994, 16.0: 0.2971413236296214, 22.0: 0.29818350530679993, 29.0: 0.2992256608717214}
1.0 0.5499999999999793
{'2022-09-30': {'0.85': 0.80860965994864, '0.7': 0.7582637425206747, '0.5': 0.7335178748438249, '0.3': 0.7157165872815109, '0.15': 0.7073502253654507, 'tte': 0.0021825396825396}}
{1

In [68]:
contract

Contract(secType='ASML', conId='SMART')

In [57]:
ib.qualifyContracts(contract)

Error 320, reqId 5: Error reading request. Unable to parse data. java.lang.NumberFormatException: For input string: "SMART", contract: Contract(secType='ASML', conId='SMART')
Unknown contract: Contract(secType='ASML', conId='SMART')


[]